In [45]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from ERFutils import path_to_ERF_outputs as output_path
from ERFutils import A
from scipy.signal import convolve

In [53]:
def convert_and_round(lat, lon, lat_values, lon_values):
    # Convert longitude from [-180, 180] to [0, 360]
    if lon < 0:
        lon += 360

    # Round latitude and longitude to nearest available values
    nearest_lat = lat_values[np.abs(lat_values - lat).argmin()]
    nearest_lon = lon_values[np.abs(lon_values - lon).argmin()]

    return nearest_lat, nearest_lon

In [52]:
# Import ERF datasets
path_to_hist = 'Streamlit/ERF_hist_En-ROADS.nc4'
ERF_hist_ds = xr.open_dataset(path_to_hist)
ERF_hist = ERF_hist_ds.ERF.values
ERF_enROADS = np.zeros(110) # Placeholder, replace with actual En-ROADS data
ERF_all = np.concatenate((ERF_hist, ERF_enROADS))

# Import Green's Functions
G_ds_path = f'{output_path}GFs/G_1pctCO2_mean_ds.nc4'
G_ds = xr.open_dataset(G_ds_path)['G[tas]']
G_ds.name = 'G[tas]'
G_ds = G_ds.rename({'year':'s'})

# Get range of available lat, lon values (grid has 1 degree
# spacing from [-89.5, 89.5], [0, 359])
lat_values = G_ds.lat.values
lon_values = G_ds.lon.values

conv_mean = False # placeholder
conv_local = True # placeholder
lat = 41.76 # placeholder
lon = -56.21 # placeholder
lat, lon = convert_and_round(lat, lon, lat_values, lon_values)

if conv_mean:
  G_ds = G_ds.weighted(A).mean(dim = ['lat','lon'])

if not conv_mean and conv_local: # placeholder
  G_ds = G_ds.sel(lat = lat, lon = lon)

GF = G_ds

length = max(len(GF.dropna(dim = 's')['s']), len(ERF_all))
if conv_mean or conv_local:
  tas_conv = convolve(np.array(GF.dropna(dim = 's')), ERF_all, 'full')[:length]

else:
  tas_conv = convolve(np.array(GF.dropna(dim = 's')), ERF_all[~np.isnan(ERF_all)][..., None, None], 'full')
  tas_conv_Glenn = {'lat' : GF.lat.values,
                    'lon' : GF.lon.values,
                    'v' : tas_conv}